In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
#load the datasets
invoices = pd.read_csv('Invoices.csv')
tanks = pd.read_csv('Tanks.csv')
locations = pd.read_csv('Locations.csv')
fuel_1 = pd.read_csv('Fuel_Level_Part_1.csv')
fuel_2 = pd.read_csv('Fuel_Level_Part_2.csv')

**Data Cleaning**

In [ ]:
invoices.columns

In [ ]:
# Renaming invoices column names
invoices= invoices.rename(columns={'Invoice Date': 'Invoice_date',
                                    'Invoice ID': 'Invoice_ID',
                                    'Invoice Gas Station Location': 'Station_location',
                                    'Gross Purchase Cost': 'Gross_purchase_cost',
                                    'Amount Purchased': 'Amount_purchased',
                                    'Fuel Type': 'Fuel_type'})
invoices.head()

In [ ]:
tanks.columns

In [ ]:
#renaming tanks column names 
tanks= tanks.rename(columns={'Tank ID': 'Tank_ID',
                               'Tank Location': 'Station_location',
                               'Tank Number':'Tank_number',
                               'Tank Type': 'Tank_type',
                               'Tank Capacity': 'Tank_capacity'})
tanks.head()

In [ ]:
#replace U with G
tanks['Tank_type'] = tanks['Tank_type'].replace({'U': 'G'})

tanks.head()

In [ ]:
locations.columns

In [ ]:
#renaming locations column names
locations= locations.rename(columns={'Gas Station Location': 'Station_location',
                                    'Gas Station Name': 'Station_name',
                                    'Gas Station Address': 'Station_address',
                                    'Gas Station Latitude': 'Station_latitude',
                                    'Gas Station Longitude': 'Station_longitude'})
locations.head()

In [ ]:
fuel_1.columns

In [ ]:
#renaming fuel_1 column names 
fuel_1= fuel_1.rename(columns={'Tank ID': 'Tank_ID',
                                'Fuel Level': 'Fuel_level',
                                'Time stamp': 'Time_stamp'})
fuel_1.head()

In [ ]:
fuel_2.columns

In [ ]:
#Renaming fuel_2 column names
fuel_2= fuel_2.rename(columns={'Tank ID': 'Tank_ID',
                               'Fuel_Level' : 'Fuel_level',
                               'Timestamp': 'Time_stamp'})
fuel_2.head()

In [ ]:
# Check for duplicates and drop them
invoices.drop_duplicates(inplace=True)
tanks.drop_duplicates(inplace=True)
locations.drop_duplicates(inplace=True)
fuel_1.drop_duplicates(inplace=True)
fuel_2.drop_duplicates(inplace=True)

In [ ]:
# Drop NaN values in fuel_1 and fuel_2 dataframes
fuel_1.dropna(inplace=True)
fuel_2.dropna(inplace=True)

In [ ]:
# Drop NaN values
invoices.dropna(inplace=True)
tanks.dropna(inplace=True)
locations.dropna(inplace=True)

In [ ]:
# Explore the data
invoices.info()
tanks.info()
locations.info()
fuel_1.info()
fuel_2.info()

**Merging dataframes**

In [ ]:
# Merge the two fuel level files
fuel_level = pd.concat([fuel_1, fuel_2])
fuel_level.head()

In [ ]:
# Merge invoices and tanks dataframes to create inventory
inventory = pd.merge(fuel_level, tanks, on='Tank_ID')
invoices['Invoice_date'] = pd.to_datetime(invoices['Invoice_date'])
inventory.head()

**Data Exploration**

In [ ]:
# Calculating the total volume of fuel purchased for each fuel type at each gas station
fuel_purchased = invoices.groupby(['Station_location', 'Fuel_type'])['Amount_purchased'].sum().reset_index()
fuel_purchased.head()

In [ ]:
# Check for outliers
sns.boxplot(x=invoices["Amount_purchased"])

In [ ]:
plt.hist(invoices['Gross_purchase_cost'])
plt.xlabel('Fuel Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.heatmap(invoices.corr(), annot=True)
plt.show()

In [ ]:
# Calculate the frequency of replenishment and total fuel purchase for each tank location 
replenishment = invoices.groupby(['Station_location', 'Fuel_type']).agg(
    {'Invoice_date': ['min', 'max', 'count'],
     'Amount_purchased': 'sum',
     'Gross_purchase_cost': 'sum'})
replenishment.columns = ['Earliest_Transaction_Date', 'Latest_Transaction_Date', 'Replenishment_Frequency', 
                           'Total_Fuel_Purchased', 'Total_Cost']

replenishment[['Earliest_Transaction_Date', 'Latest_Transaction_Date']] = replenishment[['Earliest_Transaction_Date', 'Latest_Transaction_Date']].apply(pd.to_datetime)
replenishment['Transaction_Date_diff'] = replenishment.apply(lambda row: (row['Latest_Transaction_Date']- row['Earliest_Transaction_Date']).days, axis=1).astype(int)
replenishment['Fuel_Purchased_perday']=replenishment['Total_Fuel_Purchased']/replenishment['Transaction_Date_diff']

replenishment.reset_index(inplace=True)
replenishment.head()

In [ ]:
# sort the data by tank ID and timestamp
fuel_level = fuel_level.sort_values(['Tank_ID', 'Time_stamp'])
fuel_level.head()

In [ ]:
# Create a data frame from a csv with monthly inflation rates of Canada from 2017-2019 found from Statistics Canada
infl_rate = pd.read_csv('canada_infl.csv', skiprows= [0])
infl_rate= infl_rate.rename(columns={'Canada': 'Month_year',
                                     'Percent': 'Inflation_rate'})

infl_rate['Month_year'] = pd.to_datetime(infl_rate['Month_year'], format='%y-%b')
infl_rate['Month_year'] = infl_rate['Month_year'].dt.strftime('%Y-%m')
infl_rate.head()

In [ ]:
# Create a new column Month_year containg the month of and year of the invoices
invoices['Month_year'] = invoices['Invoice_date'].dt.strftime('%Y-%m')
invoices.dropna()

In [ ]:
# Merge the inflation dataframe with invoices to adjust the Gross Purchase Cost column for each invoice based on the inflation rate of the month it was purchased
invoices_infl = pd.merge(invoices, infl_rate, on='Month_year')
invoices_infl['Adjusted_fule_price'] = invoices_infl['Gross_purchase_cost'] /invoices_infl['Amount_purchased']/ (1 + invoices_infl['Inflation_rate'])
invoices_infl.head()

In [ ]:
invoices_infl.dropna()

In [ ]:
#Overview of fule price in a week for each type in each location
# Define the station locations and fuel types
station_locations = range(1, 9)
fuel_types = ['D', 'G']
station_colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'brown', 'gray']

# Create a subplot for each fuel type
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(16, 16))

for i, fuel_type in enumerate(fuel_types):
    ax = axes[i]
    ax.set_title(fuel_type + ' Fuel')

    for j, station_location in enumerate(station_locations):
        # Filter the data for the current station location and fuel type
        data = invoices_infl[(invoices_infl.Station_location == station_location) & (invoices_infl.Fuel_type == fuel_type)]
        data_copy = data.copy()
        data_copy['Weekday'] = data_copy['Invoice_date'].dt.weekday.map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

        # Compute the mean adjusted fuel price for each weekday
        mean_price = data_copy.groupby('Weekday')['Adjusted_fule_price'].mean()

        # Plot the folding line chart for the current station location and fuel type
        sns.lineplot(x=mean_price.index, y=mean_price.values, color=station_colors[j], label='Station ' + str(station_location), ax=ax)

    # Set the axis labels
    ax.set_xlabel('Weekday')
    ax.set_ylabel('Adjusted Fuel Price')

    # Add a legend to the plot
    ax.legend()

# Show the plot
plt.show()

In [ ]:
# Tank capacity analysis
tank_capacities = inventory.groupby(['Station_location', 'Tank_ID']).agg({'Tank_capacity': 'max'}).reset_index()
average_fuel_volume = invoices.groupby(['Station_location']).agg({'Amount_purchased': 'mean'}).reset_index()
merged_tank_data = pd.merge(tank_capacities, average_fuel_volume, on=['Station_location'])
merged_tank_data['Utilization'] = merged_tank_data['Amount_purchased'] / merged_tank_data['Tank_capacity']
sns.barplot(x='Station_location', y='Utilization', hue='Tank_ID', data=merged_tank_data)
plt.legend(bbox_to_anchor=(1.05, 0.5), loc='center left')

In [ ]:
#Best day to order fuel
invoices['Weekday'] = invoices['Invoice_date'].dt.weekday.map({0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'})
sns.boxplot(x='Weekday', y='Amount_purchased', data=invoices)

**Business Question: How to order to minimize the cost and maximize the discount?**

In [ ]:
def optimal_replenishment(quantity):
    if quantity < 15000:
        discount = 0
    elif 15000 <= quantity < 25000:
        discount = 0.02
    elif 25000 <= quantity < 40000:
        discount = 0.03
    else:
        discount = 0.04
    return discount


In [ ]:
def days_to_highest_discount(quantity):
    if quantity < 25000:
        days = (25000 - quantity) / quantity
    elif quantity < 40000:
        days = (40000 - quantity) / quantity
    else:
        days = 0
    return days


In [ ]:
# Group the data by gas station, fuel type, and date, then sum the purchase quantities
replenishment_data = invoices.groupby(['Station_location', 'Fuel_type', 'Invoice_date']).agg(
    {'Amount_purchased': 'sum',
     'Gross_purchase_cost': 'sum'})

replenishment_data.reset_index(inplace=True)
replenishment_data['Invoice_date'] = pd.to_datetime(replenishment_data['Invoice_date'])

# Calculate the average number of days between replenishments for each gas station and fuel type
replenishment_data['days_diff'] = replenishment_data.groupby(['Station_location', 'Fuel_type'])['Invoice_date'].diff().dt.days
replenishment_frequency = replenishment_data.groupby(['Station_location', 'Fuel_type'])['days_diff'].mean().reset_index()

# Calculate the average replenishment quantity for each gas station and fuel type
replenishment_quantity = replenishment_data.groupby(['Station_location', 'Fuel_type'])['Amount_purchased'].mean().reset_index()

# Merge replenishment frequency and quantity data
replenishment_summary = pd.merge(replenishment_frequency, replenishment_quantity, on=['Station_location', 'Fuel_type'])

# Apply the optimal_replenishment function to the average replenishment quantity
replenishment_summary['discount'] = replenishment_summary['Amount_purchased'].apply(optimal_replenishment)

# Calculate the total cost for each gas station and fuel type based on the optimal replenishment quantity
replenishment_summary['total_cost'] = replenishment_summary['Amount_purchased'] * (1 - replenishment_summary['discount'])

replenishment_summary['Days_to_highest_discount'] = replenishment_summary['Amount_purchased'].apply(days_to_highest_discount)

replenishment_summary['Optimal_Frequency'] = replenishment_summary['days_diff'] / (1 + replenishment_summary['Days_to_highest_discount'])
replenishment_summary['Potential_Improvement'] = (replenishment_summary['days_diff'] - replenishment_summary['Optimal_Frequency']) / replenishment_summary['days_diff']

replenishment_summary.head()


In [ ]:
# Print out the optimal replenishment frequency and potential improvement for each gas station and fuel type
print("Optimal replenishment frequency and potential improvement:")
for index, row in replenishment_summary.iterrows():
    station = row['Station_location']
    fuel_type = row['Fuel_type']
    frequency = round(row['Optimal_Frequency'], 2)
    improvement = round(row['Potential_Improvement'] * 100, 2)

    print(f"For station {station} and fuel type {fuel_type}, "
          f"the optimal replenishment frequency is {frequency} days "
          f"with a potential improvement of {improvement}%.")

In [ ]:
# Best date value
invoices['Weekday'] = invoices['Invoice_date'].dt.day_name()
weekday_means = invoices.groupby('Weekday')['Amount_purchased'].mean().reset_index()
weekday_means = weekday_means.sort_values('Amount_purchased', ascending=False)
max_day_value = weekday_means.iloc[0]['Weekday']
print("The day with highest average amount of fuel purchased is:", max_day_value)

**Business Question: What day to order?**

In [ ]:
# Best Date Price
station_locations = range(1,9)  # List of station locations to consider
fuel_types = ['D', 'G']  # List of fuel types to consider
min_prices_D = float('inf')  # Initialize the lowest price for type D fuel to infinity
min_prices_G = float('inf')  # Initialize the lowest price for type G fuel to infinity

for i, station_location in enumerate(station_locations):
    for j, fuel_type in enumerate(fuel_types):
        # Filter the data for the current station location and fuel type
        data = invoices_infl[(invoices_infl.Station_location == station_location) & (invoices_infl.Fuel_type == fuel_type)]
        data_copy = data.copy()
        data_copy['Weekday'] = data_copy['Invoice_date'].dt.weekday.map({0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'})

        weekday_means = data_copy.groupby('Weekday')['Adjusted_fule_price'].mean().reset_index()
        weekday_means = weekday_means.sort_values('Adjusted_fule_price', ascending=True)
        best_day_value = weekday_means.iloc[0]['Weekday']
        best_day_price = weekday_means.iloc[0]['Adjusted_fule_price']
        print(f"The best date price of Type {fuel_type} fuel in the station {station_location} is {best_day_value} with a price of {best_day_price:.2f}")
        
        min_price = weekday_means.iloc[0]['Adjusted_fule_price'].min()        
        # Update the lowest price for the current fuel type
        if fuel_type == 'D' and min_price < min_prices_D:
            min_prices_D = min_price
            min_price_D_date = best_day_value
            min_price_D_station = station_location
        elif fuel_type == 'G' and min_price < min_prices_G:
            min_prices_G = min_price
            min_price_G_date = best_day_value
            min_price_G_station = station_location

print(f"The lowest price for type D fuel is {min_prices_D:.2f} in station {min_price_D_station} on {min_price_D_date}")
print(f"The lowest price for type G fuel is {min_prices_G:.2f} in station {min_price_G_station} on {min_price_G_date}")

**Business Question: Identify which fuel stations would benefit most.**

In [ ]:
# Group inventory by station and date
invoices_grouped = invoices.groupby(['Station_location', 'Invoice_date'])

# Calculate total fuel purchased and total cost for each group
total_fuel_purchased = invoices_grouped['Amount_purchased'].sum()
total_cost = invoices_grouped['Gross_purchase_cost'].sum().round(2)

# Calculate average fuel price for each group
avg_price = (total_cost / total_fuel_purchased).round(2)


# Calculate inventory change for each group
inventory_change = invoices_grouped['Amount_purchased'].sum().diff().fillna(0)

# Combine all results into a single DataFrame
inventory_analysis = pd.DataFrame({'Total_fuel_purchased': total_fuel_purchased,
                                   'Total_cost': total_cost,
                                   'Average_price': avg_price,
                                   'Inventory_change': inventory_change})

# Print the first five rows of the analysis
inventory_analysis.head()

In [ ]:
# Calculate average daily fuel consumption for each gas station and fuel type
daily_consumption = invoices.groupby(['Station_location', 'Fuel_type', 'Invoice_date'])['Amount_purchased'].sum().reset_index()
daily_consumption['Day_of_Week'] = daily_consumption['Invoice_date'].dt.day_name()

# Calculate average daily consumption for each gas station and fuel type
daily_consumption['Avg_Daily_Consumption'] = daily_consumption.groupby(['Station_location', 'Fuel_type', 'Invoice_date'])['Amount_purchased'].transform('mean')

# Calculate the daily change in consumption for each gas station and fuel type
daily_consumption['Daily_Change'] = daily_consumption.groupby(['Station_location', 'Fuel_type'])['Avg_Daily_Consumption'].diff()

daily_consumption.head()


In [ ]:
# Calculate the potential savings using the optimal discount
invoices['Potential_Cost'] = invoices['Amount_purchased'] * (1 - invoices['Amount_purchased'].apply(optimal_replenishment))
invoices['Potential_Cost'] = invoices['Potential_Cost'].round(2)

# Group the data by gas station and fuel type
potential_savings = invoices.groupby(['Station_location', 'Fuel_type'])['Potential_Cost'].sum().reset_index()
actual_cost = invoices.groupby(['Station_location', 'Fuel_type'])['Gross_purchase_cost'].sum().reset_index()

# Merge the potential savings and actual cost
performance_df = potential_savings.merge(actual_cost, on=['Station_location', 'Fuel_type'])

# Calculate the actual savings and savings percentage
performance_df['Actual_Savings'] = performance_df['Potential_Cost'] - performance_df['Gross_purchase_cost'].round(2)

performance_df['Savings_Potential'] = performance_df['Gross_purchase_cost'] - performance_df['Potential_Cost']
performance_df['Savings_Potential'] = performance_df['Savings_Potential'].round(2)

performance_df['Savings_Percentage'] = (performance_df['Savings_Potential'] / performance_df['Potential_Cost']).round(2)
performance_df.head()


In [ ]:
print("Maximum Possible Savings:")
for index, row in performance_df.iterrows():
    station = row['Station_location']
    fuel_type = row['Fuel_type']
    savings_potential = row['Savings_Potential']
    savings_percentage = row['Savings_Percentage'] * 100  # Convert to percentage

    print(f"For station {station} and fuel type {fuel_type}, "
          f"the maximum possible savings that could be achieved if the gas station always purchased fuel at the highest discount rate is up to ${savings_potential:.2f} "
          f"in gross purchase costs per year. This would result in a {savings_percentage:.2f}% savings.")


In [ ]:
#Identify which fuel station and which tank benefit most
performance_df.sort_values(by='Savings_Percentage', axis=0, ascending=False, inplace=True)
performance_grouped=performance_df.groupby(['Station_location'])[['Savings_Potential','Potential_Cost']].sum().reset_index()
performance_grouped['Savings_Percentage']= (performance_grouped['Savings_Potential'] / performance_grouped['Potential_Cost'])  
performance_grouped.sort_values(by='Savings_Percentage', axis=0, ascending=False, inplace=True)
print(f'Fuel station {performance_df.Station_location[8]} type {performance_df.Fuel_type[8]} would benefit most, with {performance_df.Savings_Percentage[0]:.2f} percentage savings.')
print(f'Fuel station {performance_grouped.Station_location[0]} would benefit most, with {performance_grouped.Savings_Percentage[0]:.2f} percentage savings.')
print(performance_grouped.head())
performance_df.head()

**Business Question: What is the optimal tank size? Should we increase the capacity of tank size?**

In [ ]:
# Develop recommendations on tanks size
def calculate_new_tank_size(utilization, current_tank_size, days_diff):
    gallons_per_day = current_tank_size * utilization / days_diff

    # Determine the optimal tank size based on the number of days between deliveries
    return gallons_per_day * days_diff

In [ ]:
def generate_recommendations(recommendations_df):
    recommendations = []
    for index, row in recommendations_df.iterrows():
        recommendation = f"For station {row['Station_location']} and fuel type {row['Fuel_type']}, "
        recommendation += f"switching to a {round(row['New_tank_size'],4)} gallon tank can save up to ${round(row['Savings_Potential'], 2)} in gross purchase costs per year. "
        recommendation += f"This would result in a {round(row['Savings_Percentage']*100, 2)}% savings."
        recommendations.append(recommendation)
    return recommendations

In [ ]:
recommendations_df = performance_df.merge(tank_capacities.groupby(['Station_location'])['Tank_capacity'].mean().reset_index(), on='Station_location')
recommendations_df['New_tank_size'] = merged_tank_data.apply(
    lambda row: calculate_new_tank_size(row['Utilization'], row['Tank_capacity'],7), axis=1)
# Sort recommendations by potential cost savings
recommendations_df = recommendations_df.sort_values('Savings_Potential', ascending=False)

In [ ]:
recommendations = generate_recommendations(recommendations_df)
print("Recommended tank sizes:")
for recommendation in recommendations:
    print(recommendation)